<a href="https://colab.research.google.com/github/ravi-annaswamy/AdversarialAudioSeparation/blob/master/Explore_Pretrained_Language_Model_Tamil_Wikipedia_8K_wordparts_FastLM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1. Imports

In [1]:
!pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (1.0.51)


In [2]:
!pip install sentencepiece

In [0]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.51', '1.0.1.post2')

## Step 2. Download Tamil LM models from Dropbox

In [18]:
!wget https://www.dropbox.com/s/zozzrgawulkwtxa/wikitalm_8k_447_third.pth

--2019-04-06 13:53:02--  https://www.dropbox.com/s/zozzrgawulkwtxa/wikitalm_8k_447_third.pth
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zozzrgawulkwtxa/wikitalm_8k_447_third.pth [following]
--2019-04-06 13:53:02--  https://www.dropbox.com/s/raw/zozzrgawulkwtxa/wikitalm_8k_447_third.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc02c52ea5453d59ae7b84032a4b.dl.dropboxusercontent.com/cd/0/inline/Aegt5FQ5FbvOYOUbIr8chSyC4L5dN4DqE3kIKB601ohRyjmlZBIbVlSjcMe-wbDZWrlqDDgBdoadbXQ4qR3zoSPq7zu8jAvtnbs9UVeaE12ywQ/file# [following]
--2019-04-06 13:53:02--  https://uc02c52ea5453d59ae7b84032a4b.dl.dropboxusercontent.com/cd/0/inline/Aegt5FQ5FbvOYOUbIr8chSyC4L5dN4DqE3kIKB601ohRyjmlZBIbVlSjcMe-wbDZWrlqDDgBdoadbXQ4qR3zoSPq7zu8jAvt

In [17]:
!wget https://www.dropbox.com/s/hl7cowdki4ngf65/data_lm_tawiki_8k.pkl

--2019-04-06 13:52:53--  https://www.dropbox.com/s/hl7cowdki4ngf65/data_lm_tawiki_8k.pkl
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/hl7cowdki4ngf65/data_lm_tawiki_8k.pkl [following]
--2019-04-06 13:52:54--  https://www.dropbox.com/s/raw/hl7cowdki4ngf65/data_lm_tawiki_8k.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd74ce3d5e1d7c5d44e6f515a59.dl.dropboxusercontent.com/cd/0/inline/AegEQ0aAEXfBYukJgEAPcX0EoTzq0-78KRwXSUn3GVQt_5UN6zXb98maTCjhXVqfB6jhzWb5S3lROy4_-yXnpzzAz-PTc-ZsigXjV9oVBZ1J8w/file# [following]
--2019-04-06 13:52:54--  https://ucd74ce3d5e1d7c5d44e6f515a59.dl.dropboxusercontent.com/cd/0/inline/AegEQ0aAEXfBYukJgEAPcX0EoTzq0-78KRwXSUn3GVQt_5UN6zXb98maTCjhXVqfB6jhzWb5S3lROy4_-yXnpzzAz-PTc-ZsigXjV9oVBZ1J

In [16]:
!wget https://www.dropbox.com/s/9q1pkf6c7ahs9qm/tamil_spm_8k.model

--2019-04-06 13:52:48--  https://www.dropbox.com/s/9q1pkf6c7ahs9qm/tamil_spm_8k.model
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9q1pkf6c7ahs9qm/tamil_spm_8k.model [following]
--2019-04-06 13:52:48--  https://www.dropbox.com/s/raw/9q1pkf6c7ahs9qm/tamil_spm_8k.model
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2989d86e3def2443cd4233272c.dl.dropboxusercontent.com/cd/0/inline/AehG8uENNhmbPoXUSXwSZX-MZD44svacyvfKo6vfUJVvh2tu_raui5wRmDj1dGyCfJSM7vnXtQNstCqlfsGQqdpXAjxlKS0yC7IC3IUO3-SCWQ/file# [following]
--2019-04-06 13:52:48--  https://uc2989d86e3def2443cd4233272c.dl.dropboxusercontent.com/cd/0/inline/AehG8uENNhmbPoXUSXwSZX-MZD44svacyvfKo6vfUJVvh2tu_raui5wRmDj1dGyCfJSM7vnXtQNstCqlfsGQqdpXAjxlKS0yC7IC3IUO3-SCWQ/file
R

In [14]:
!wget https://www.dropbox.com/s/5u4l67faq46jbvw/tamil_spm_8k.vocab

--2019-04-06 13:52:31--  https://www.dropbox.com/s/5u4l67faq46jbvw/tamil_spm_8k.vocab
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5u4l67faq46jbvw/tamil_spm_8k.vocab [following]
--2019-04-06 13:52:31--  https://www.dropbox.com/s/raw/5u4l67faq46jbvw/tamil_spm_8k.vocab
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4aea5a4e4b7104931435c22897.dl.dropboxusercontent.com/cd/0/inline/AeiMh4RkoKW_Gdlc2aytqpYmS0-c8Njg4lU1SscAj11jl6t3cbxS-JAsAMNwTZWmiqrk0nsTaA9FEKeauoF8STXiiKbq7eoAcPyBfhKKpoB4HA/file# [following]
--2019-04-06 13:52:32--  https://uc4aea5a4e4b7104931435c22897.dl.dropboxusercontent.com/cd/0/inline/AeiMh4RkoKW_Gdlc2aytqpYmS0-c8Njg4lU1SscAj11jl6t3cbxS-JAsAMNwTZWmiqrk0nsTaA9FEKeauoF8STXiiKbq7eoAcPyBfhKKpoB4HA/file
R

In [34]:
!wget https://www.dropbox.com/s/24nq1vefalhkjjv/inltk.zip

--2019-04-06 13:56:42--  https://www.dropbox.com/s/24nq1vefalhkjjv/inltk.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/24nq1vefalhkjjv/inltk.zip [following]
--2019-04-06 13:56:42--  https://www.dropbox.com/s/raw/24nq1vefalhkjjv/inltk.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc210b434dd1e502ae61e3e1c9cc.dl.dropboxusercontent.com/cd/0/inline/AehPNdVh3kKxu6unMBTJv63rDOytbKy7Rj9MbfLj3o144t4smujl13cLNfstlg9Qg4SDkexz9J88q7F1K-6h_uAtekrmFQZVZx7EhPvatN48Yw/file# [following]
--2019-04-06 13:56:42--  https://uc210b434dd1e502ae61e3e1c9cc.dl.dropboxusercontent.com/cd/0/inline/AehPNdVh3kKxu6unMBTJv63rDOytbKy7Rj9MbfLj3o144t4smujl13cLNfstlg9Qg4SDkexz9J88q7F1K-6h_uAtekrmFQZVZx7EhPvatN48Yw/file
Resolving uc210b434dd1e502ae

In [35]:
!unzip inltk.zip

Archive:  inltk.zip
   creating: inltk/
  inflating: inltk/.DS_Store         
   creating: __MACOSX/
   creating: __MACOSX/inltk/
  inflating: __MACOSX/inltk/._.DS_Store  
   creating: inltk/__pycache__/
  inflating: inltk/__pycache__/.DS_Store  
   creating: __MACOSX/inltk/__pycache__/
  inflating: __MACOSX/inltk/__pycache__/._.DS_Store  
  inflating: inltk/__pycache__/tokenizer.cpython-36.pyc  
 extracting: inltk/init.py           
  inflating: __MACOSX/inltk/._init.py  
  inflating: inltk/tokenizer.py      
  inflating: __MACOSX/inltk/._tokenizer.py  


In [36]:
!ls

data_lm_tawiki_8k.pkl  inltk.zip  sample_data	      tamil_spm_8k.vocab
inltk		       __MACOSX   tamil_spm_8k.model  wikitalm_8k_447_third.pth


In [43]:
!mkdir models
!mv wikitalm_8k_447_third.pth models/

mkdir: cannot create directory ‘models’: File exists
mv: cannot stat 'wikitalm_8k_447_third.pth': No such file or directory


## Step 3. Setup the tokenizer

In [0]:
torch.cuda.set_device(0)

In [0]:
path = Path('.')

In [0]:
class TamilTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str("tamil_spm_8k.model"))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [0]:
sp = spm.SentencePieceProcessor()
sp.Load(str('tamil_spm_8k.model'))
itos = [sp.IdToPiece(int(i)) for i in range(8000)]

In [0]:
tamil_vocab = Vocab(itos)

In [0]:
tokenizer = Tokenizer(tok_func=TamilTokenizer, lang='ta')

In [32]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

## Step 4. Load the Data

In [0]:
data_lm = load_data('.', 'data_lm_tawiki_8k.pkl')

In [38]:
data_lm.show_batch()

idx,text
0,"▁c ric k et <unk> ▁a s s o ci ation <unk> ▁s ta d ium ) ▁என்பது ▁ராஜ் கோட் , இந்தியா வில் ▁உள்ள ▁துடுப்பாட்ட ▁அரங்க ம் ▁ஆகும் . ▁இது ▁இது ▁காந்த ேரி ▁துடுப்பாட்ட ▁அரங்க ம் ▁எனவும் ▁ அழைக்கப்படுகிறது ) ▁ , ▁இது ▁குசராத் து ▁மாநிலத்தின் ▁முதல் ▁சூரிய ▁ஆற்றல் ▁அரங்க ம் ▁ஆகும் . ▁நவம்பர் ▁ , ▁2015 ▁இல் ▁இந்த ▁அரங்க ம் ▁உட்பட ▁இந்தியாவில் ▁உள்ள ▁ஆறு ▁புதிய ▁துடுப்பாட்ட ▁அரங்க ு கள்"
1,"▁ / ▁ wiki ? curi d = 21 08 22 "" ▁title ="" தமிழ் ▁மொழியில் ▁உள்ள ▁பேச்சு ▁வழக்கில் ▁திரி ந்த ▁தமிழ்ச் ▁சொற்கள் ""> ▁தமிழ் ▁மொழியில் ▁உள்ள ▁பேச்சு ▁வழக்கில் ▁திரி ந்த ▁தமிழ்ச் ▁சொற்கள் ▁தமிழ் ▁மொழியில் ▁உள்ள ▁சில ▁சொற்கள் ▁காலப்போக்கில் ▁பேச்சு ▁வழக்கில் ▁பொருள் ▁திரி ந்து ▁எழுத்துக்கள் ▁திரி ந்து ▁உள்ளன . ▁அவற்றின் ▁தொகுப்பு . ▁< ▁ / ▁ doc > ▁< doc ▁id ="" 21 08 23"" ▁ url ="" https :"
2,"சி க் மக ளூர் ""> ▁சி க் மக ளூர் ▁சி க் மக ளூர் ▁("" <unk> ch ick ma g al ur "") ▁இந்தியாவின் ▁கர்நாடகா ▁மாநிலத்தில் ▁சி க்க ம கள ூ ரு ▁மாவட்டத்தில் ▁அமைந்துள்ள ▁ஒரு ▁நகரம் . ▁குளம் பி க்கு ▁( co ff ee ) ▁புகழ்பெற்ற ▁மு ல்ல யா நகர ி ▁மலைத்தொடர் ▁அடி வார த்தில் ▁சி க் ம ங்கள ூர் ▁அமைந்துள்ளது , ▁இது ▁கர்நாடகா வின் ▁குளம் பி ▁விளை யும் ▁நில மாக"
3,"▁1 20 ▁மில்லி ▁ஆம் பியர் ▁வரையிலும் ▁உள்ளது . ▁இவ்விரு ▁மின்னோட்ட ங்களும் ▁தனித்தனி யான வை யான ாலும் ▁ஒன்ற ி ற்க ொன்று ▁தொடர்புடைய ன . ▁இவை களை ▁இணைக்கும் ▁ஒரு ▁காரணி ▁எதிர்மின் ▁முனை க்கரு கில் ▁காணப்படும் ▁இட ▁மின்ன ூட்ட மாகும் ▁("" s p ace ▁c har ge "") . ▁குழாய ில் ▁இரு மின் ▁முனை களுக்கு முள்ள ▁மின் ▁அழுத்த ▁வேறுபாடு ▁( p . d ) ▁குறைவாக ▁உள்ள போது , ▁இழை யிலிருந்து ▁இலத்திரன் கள் ▁வி"
4,ங்கள் ▁இடம்பெற்ற ுள்ள ▁ஒரு ▁சம் பிர தா ய ▁மத மாக ▁இருந்தது . ▁இதனால் ▁ஓவிய ம் ▁மற்றும் ▁சிற்பங்கள் ▁உள்ள ▁பட ங்கள் ▁உட்பட்ட ▁மத ப் ▁பொருட்களை ▁உருவாக்கும் ▁ஊக்க ம் ▁உ ண்ட ானது . ▁1930 ▁ களில் ▁ஸ்டா லி னின் ▁சீர்திருத்த க் ▁கருத்து களுக்குப் ▁பிறகு ▁பௌத்த ம் ▁மற்றும் ▁மத குரு ▁நம்பிக்கை கள் ▁மங்கோலிய ▁மக்கள் ▁குடியரச ில் ▁செல்வ ாக்க ிழந்த ன . ▁உட்புற ▁மங்கோலிய ாவில் ▁பாரம்பரிய ▁மத க் கொள் கை கள் ▁பாதிக்கப்பட்ட ு ▁கலாச்சார


In [39]:
len(data_lm.vocab.itos)

8000

## Step 4. Load pretrained model

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3, pretrained=False)

In [0]:
learn.load('wikitalm_8k_447_third', with_opt=True);

## Step 5. Generate sample text from learned model.

In [0]:
def create(text, spm=False, N_WORDS=50, N_SENTENCES=4):
    TEXT=sp.EncodeAsPieces(text)
    TEXT=' '.join(TEXT)
    sp_merge=lambda s:s.replace(' ','').replace('▁',' ')
    samples=[]
    for _ in range(N_SENTENCES):
      sample=learn.predict(TEXT, N_WORDS, temperature=0.7)
      samples.append(sample)
      if spm:
          print(sample)
          print()
    if spm:
        print()
    for sample in samples:
      sample=sp_merge(sample)
      print(sample)
      print()

In [46]:
text='17-ஆம் நூற்றாண்டில்'
create(text, spm=True)

▁17 - ஆம் ▁நூற்றாண்டில் ▁மற்றும் ▁ஆர் த ரிய ப் ▁போ ப்பா ண்ட வர் களை ▁ தற் செயல ாகக் ▁கண்டறிந்த ு ▁நேர் ந்த தாக ▁கருதப்படுகிறது . ▁இது ▁குறித்த ▁ஆய்வு களின் ம் ▁படி வளர்ச்சி க் ▁கால த்தை ▁அடிப்படையாகக் ▁கொண்ட தாகவும் , ▁குறிப்பாக , ▁புதிய ▁சமுதாய ம் ▁மற்றும் ▁புற ப் பரப்பு களின் ▁நடு வே ▁சமய ப் ▁பரப்பு

▁17 - ஆம் ▁நூற்றாண்டில் ▁உருவாக்கப்பட்ட து . ▁இந்த ▁ ரை சின் ▁குடும்ப த்தா ரால் ▁மேற்கொள்ள ப்படும் ▁ஒரு ▁புதிய ▁முயற்சி யானது ▁கடந்த ▁10 ▁ஆண்டுகளில் யிர ான் , ▁கால ▁நிறைவு க் ▁காலம் , ▁மற்றும் ▁முன் னோ க்கி ச் ▁சென்ற ுள்ளன ▁என்று ▁கூறப்படுகிறது . ▁வேறு ▁சில ▁பள்ளி களில் ▁இருந்து ம் , ▁பல ▁இடங்களுக்கு ▁ஒரு ▁தனி ச் சிறப்ப

▁17 - ஆம் ▁நூற்றாண்டில் , ▁" " v il le y " " ▁என்ற ▁பெயரில் ▁" " நான் ▁செய்ய க் ▁கூடிய ▁நீ ரி ழிவு " " ▁என்று ▁அதன் ▁பெயரை ▁மாற்றி க் ▁கொண்டு , ▁அதற்கு ▁' க ங் ரீ னா ' ▁எனப் ▁பெயரிட்ட ார் . ▁அது ▁“ அ ன் பு ▁நீ ங்கள் ” ▁என ▁குறிப்பிட

▁17 - ஆம் ▁நூற்றாண்டில் ▁” சா ரா ” , ▁“ இரு ம்பு ” , ▁' பே ரி ஸ் ' , ▁“ சூ ரியன் ” , ▁“ ஆ ல் ’ , ▁“ பி ரை யர் ” , ▁“ பி ரியா ஸ் ” , ▁” பூ ஸ

In [47]:
text='தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர்'
create(text)

 தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர், மன்னர் முதலியோர் இருந்தனர். < / doc> <doc id="212709" url="https: / / ta.wikipedia.org / wiki?curid=213209" title="வே

 தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர் முர்ரே வின்சன்ட் மற்றும் அவரது மனைவி பிரவுன் ஆகியோரின் சகோதரன் சதியாவிடமிருந்து விடுதலை பெற்று, மன்சப்தாரியின் இடத்தைப் பிடிக்காமல் நடந்து கொள்ளவில்லை. ஆண்களை பெண்களுக்குக் கொண்டு சென்ற போது, மீண்டும் ஆணுறை

 தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர் மன்ஹாட்டன் ட்ரெஷ்வாவை இடித்துவிட்டு, லைபீரியாவுக்கு வெளியே, லின்டாக், வெர்ஜின், ஸ்கைஸ்காடன், கோடாஸ்ட், ட்ரோலிங், லிட்டில் டேல்

 தனது ஆட்சிக் காலத்தில் பிரான்சின் அரசுத்தலைவர் எட்வர்டு மார்ட்டின் பிரபுவின் ஆட்சியின் கீழ் வந்தது. இதன் போது இந்த யுத்தத்தின் போது, பதினேழாம் நூற்றாண்டில், மானிடர்களின் வளர்ச்சியை தடுக்க பல திட்டங்களை வகுத்தது. இச்சபையின் நிறுவனர் கணிசமாக இருந்தார். இதன்



In [48]:
text='இன்று இவர் பிரான்ஸ், ஜேர்மனி ஆகிய'
text='இவரது ஆக்கங்கள் பல்வேறு'
create(text)

 இவரது ஆக்கங்கள் பல்வேறுபட்ட மூலக்கூற்றில் உள்ளடக்கப்பட்டுள்ளன. < / doc> <doc id="191495" url="https: / / ta.wikipedia.org / wiki?curid=191498" title

 இவரது ஆக்கங்கள் பல்வேறு பொருட்கள்,பட்டியல்கள், அணிகலன்கள், வெற்றிகிணறுகள், பல்வகையான வாகனங்கள், பண்ணைகள், பாறைகள் மற்றும் நகைகள் ஆகியவற்றைப் பயன்படுத்திக்கொண்டுன. இதுபோன்று, பருவம் மற்றும் வருடாந்திர நாணயங்கள்

 இவரது ஆக்கங்கள் பல்வேறுபட்ட விவரக் கூற்றுகளுடன் இணைக்கப்பட்டுள்ளன. அவை: < / doc> <doc id="191294" url="https: / / ta.wikipedia.org / wiki?curid=191294

 இவரது ஆக்கங்கள் பல்வேறு வகையிலும் உள்ளன. ஒருசிலரும் இவ்வளிமையில் தங்கள் உறவுகளை வெளிப்படுத்திக் கொள்கிறார்கள். இவர்களைக் கொண்டு அறியப்படவில்லை. இருந்தும் அவர்களுக்காக சவாலுக்குரிய மிக உயரிய விருதான பத்மசிறீ விருது வழங்கி சிறப்பிக்கப்படுகின்றது.



## Step 6. Prompt it to create entire article segments from scratch.

In [49]:
sp_merge=lambda s:s.replace(' ','').replace('▁',' ')
sp_merge('<doc id="91202" url="https: / / ta.wikipedia.org / wiki?')

'<docid="91202"url="https://ta.wikipedia.org/wiki?'

In [50]:
create('<doc id="52700" url="https://ta.wikipedia.org/wiki?', N_WORDS=200, N_SENTENCES=10)

 <doc id="52700" url="https://ta.wikipedia.org/wiki? www.twofa.com / tamil / article107904.html < / doc> <doc id="160883" url="https: / / ta.wikipedia.org / wiki?curid=160883" title="பொருளாதாரம்"> பொருளாதாரம் பொருளாதாரம் என்பது உற்பத்தி மற்றும் உற்பத்தியின் ஒரு உற்பத்தி ஆகும். அது உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, மேலாண்மை, உற்பத்தி, விநியோகம் என பல துறைகளிலும் உற்பத்தி செய்யப்படுகின்றன. அடிப்படை உற்பத்தியாளர்களின் எண்ணிக்கை, உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி மற்றும் உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, உற்பத்தி, நுகர்வோர் மற்றும் வேளாண்மை, உற்பத்தி, உற்பத்தி, தொழில்துறை, பானம் ஆகியவற்றைப் பயன்படுத்துகிறது. < / doc> <doc id="81413" url="https: / / ta.wikipedia.org / wiki?curid=82613" title

 <doc id="52700" url="https://ta.wikipedia.org/wiki? / http: / / www.news.com / national / films / 2016 / 08 / blog-post_10.html < / doc> <doc id="190598" url="https: / / ta.wikipedia.org / wiki?curid=190594" title="உயிரியல்"> உயிரியல் உயிரியல் (modology) என்பது, ஒர

## I am amazed by these two sentences on chemical reaction: 
ஒரு பொருள் வெப்பம் அடைந்து வேதிப்பொருளாக வினைபுரிகிறது. 
வினையின் இறுதி வினையில் வினையின் விளைவுகள் தோன்றுகின்றன.

In [0]:
# Following code is adapted from: https://github.com/cstorm125/thai2fit/blob/master/thwiki_lm/thwiki_lm.ipynb

In [61]:
#how we extracted the embeddings
emb_weights = list(learn.model.named_parameters())[0][1]
emb_np = to_np(emb_weights.data)
emb_np.shape

(8000, 400)

In [0]:
import pandas as pd

In [63]:
tamilvec = pd.DataFrame(emb_np)
tamilvec.index = data_lm.vocab.itos
print(tamilvec.shape)
tamilvec.tail(10)

(8000, 400)


,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
▁இருபத்தி,-1.077665,-0.390356,0.163234,0.283401,-1.093594,0.503261,0.321981,0.401306,0.076390,0.727045,...,-0.325647,-0.522335,-0.188289,-0.356652,-0.485064,-0.121918,-0.449520,-0.116807,-0.113558,0.338918
▁சந்தர்ப்ப,-0.121100,-0.049716,-0.167197,0.365919,-1.635901,1.082761,-0.331799,-0.512262,-0.723081,0.198193,...,-0.447440,0.219738,-0.426128,-0.254355,0.416040,0.167318,-0.591591,0.540607,-0.062622,0.472127
சமூக,0.209300,0.599686,0.039264,0.364590,-1.840876,0.746350,0.312119,-0.538757,-1.021886,-0.038306,...,-0.784274,-0.125212,-0.364481,-0.409861,-1.035161,0.422461,-0.901599,0.352806,0.328277,0.911684
▁வற்புறுத்த,-0.426133,0.192765,0.032120,-0.392156,-1.239767,0.556953,0.268325,-0.056107,-1.168194,0.245597,...,-0.444652,-0.382560,-0.342674,-0.075639,-0.399160,0.054154,-0.710152,0.367050,0.494992,0.724223
▁கூகுள்,-0.485771,0.094991,0.122683,0.529538,-1.633889,0.722075,-0.180137,0.111167,-1.436899,0.640899,...,-0.305726,-0.358842,0.399187,-0.595666,-0.730073,-0.012862,-0.467213,0.846618,-0.061463,-0.000046
▁பிரசித்தி,-0.039581,-0.460073,0.109150,0.130076,-1.213040,0.653963,-0.115663,-0.748029,-1.023990,0.631861,...,-0.744943,-0.007287,0.159883,-0.335717,-1.102930,-0.095574,-0.667286,0.495898,0.475527,-0.187048
▁சீக்கியர்கள,-0.457573,-0.029164,0.267646,0.384505,-0.755809,0.871107,0.213756,0.196102,-1.500173,0.445593,...,-0.522237,0.404235,-0.183606,-0.075815,-0.504381,-0.235018,-0.947623,0.127727,0.167060,-0.604006
▁மத்திமம்,-0.537851,0.323388,-0.127235,0.247344,-1.732942,0.839073,-0.374221,-0.406411,-0.810076,0.052043,...,0.429243,-0.557794,-0.199908,-0.246132,-0.220002,0.181184,-0.230351,-0.076691,0.397272,0.882706
“,-0.487057,0.101350,0.323846,0.379314,-1.294675,0.587669,0.423591,-0.214962,-1.348075,0.608770,...,-0.735086,-0.604390,0.388983,-0.206811,-1.178709,-1.793508,-0.123695,0.008620,-0.169480,0.077503
‘,-1.062528,-0.326936,-0.282349,0.499380,-1.066751,0.758417,-0.177993,-0.499733,-1.039507,0.274381,...,-0.243882,-0.269090,0.277599,-0.375555,-1.238790,0.933866,0.082637,-0.540410,0.465956,0.394246


In [0]:
tamilvec.to_csv('tamilvec_wiki8k.csv')

## Exploring Word vectors

In [0]:
word2index = tamilvec.index
wordvec = dict(zip(tamilvec.index, emb_np))


In [80]:
wordvec['▁பல்கலைக்கழகம்']

array([-0.203724,  0.020787,  0.32179 , -0.130676, ..., -0.058546, -0.193656,  0.322165, -0.799783], dtype=float32)

In [0]:
from collections import Counter
import math 

def similar(target='▁பல்கலைக்கழகம்', N=15):
    scores = Counter()
    for word in word2index:
        raw_difference = wordvec[word] - (wordvec[target])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(N)

In [92]:
similar('▁பல்கலைக்கழகம்')

[('▁பல்கலைக்கழகம்', -0.0),
 ('▁பல்கலைக்கழக', -7.622758631724941),
 ('▁பல்கலைக்', -8.381474130733048),
 ('▁அருங்காட்சியகம்', -9.934804107363593),
 ('▁கல்லூரி', -10.041136493606821),
 ('▁நூலகம்', -10.318009302485999),
 ('▁பாடசாலை', -10.527922100302066),
 ('▁பள்ளி', -10.59030479469354),
 ('▁உயர்நிலைப்பள்ளி', -10.687967829421753),
 ('▁கழகம்', -10.696278333915997),
 ('▁நிறுவனம்', -10.705079108673019),
 ('▁நூலக', -10.819976809333246),
 ('▁பேரரசு', -10.837515571472192),
 ('▁பொறியியல்', -10.891923176733965),
 ('▁தேவாலய', -10.900616360515421)]

In [0]:
import random
def random_word_similarity():
  word = random.choice(word2index)
  print('Words most similar to:', word, 'are:')
  for w,d in similar(word):
    print(w,d)

In [90]:
random_word_similarity()

Words most similar to: ▁நடத்தை are:
▁நடத்தை -0.0
▁இயல்ப -8.215403268017466
▁சூழ்நிலை -8.370106800581947
▁இயல்பான -8.373966557268977
▁இயல்பு -8.431248540869035
▁மரபியல் -8.458122979790913
▁சுற்றுச்சூழல் -8.48872842641038
▁எதிர்கால -8.503666136174594
▁வடிவமைப்பு -8.514070598349551
▁உளவியல் -8.525539534719877
▁உணர்ச்சி -8.57813139832995
▁ஒழுக்க -8.620055584636072
▁செயல்பாடு -8.622697410599256
▁நிலைமை -8.707858288932846
▁செய்முறை -8.712483718098287


In [99]:
random_word_similarity()

Words most similar to: ▁சங்கத்தின் are:
▁சங்கத்தின் -0.0
▁சங்கம் -7.752018149978662
▁கழகத்தின் -8.312731243755607
▁கழகத்தில் -9.466042169985057
▁குழுவின் -9.614987310117067
▁கழக -9.781332243940135
▁நிறுவனத்தின் -10.006658241555689
▁கூட்டமைப்ப -10.037624172912833
▁கழகம் -10.144654668297827
▁கூட்டமைப்பு -10.158870584535885
▁அமைப்பின் -10.187481739690915
▁கவுன்சில -10.188254483101943
▁பேரவை -10.194472010018304
▁கட்சியின் -10.307315483179591
▁செயலாளராக -10.32021642437314


In [100]:
for _ in range(10):
  random_word_similarity()
  print()

Words most similar to: ▁செயற்பாடு are:
▁செயற்பாடு -0.0
▁செயல்பாடு -6.688972917360301
▁செயல்முறை -8.375478185814748
▁சிக்கல் -8.571231007726936
▁செயல்பாட்ட -8.62956489892192
▁நடவடிக்கைகள் -8.792229348100095
▁செயற்பாட்ட -8.821875366880704
▁செயல் -8.86636707921065
▁கட்டமைப்பு -8.89881943842484
▁செய்முறை -8.961576606589867
▁வழிமுறை -8.98832087845033
▁படிம -9.001573100707782
▁கோட்பாடு -9.013804171911644
▁சடங்கு -9.023201609420761
▁சிந்தனை -9.09664756256174

Words most similar to: ▁உருவாகிறது are:
▁உருவாகிறது -0.0
▁ஏற்படுகிறது -9.886452095247327
▁கிடைக்கிறது -10.153520540378487
▁உருவானது -10.24272964378204
▁உருவாகின்றன -10.4695628410069
▁தோன்றுகிறது -10.525247609258422
▁உருவாகிய -10.626830240583418
▁உண்டாகும் -10.867253533946046
▁உருவாகி -10.995676604988713
▁நிகழ்கிறது -11.054011713806595
▁ஏற்படலாம் -11.062169234277079
▁உருவாகும் -11.218150897684284
▁உருவாக -11.3653929722661
▁அதனுடைய -11.374468075125659
▁உருவாவத -11.377920080351405

Words most similar to: ▁கருதப்பட்ட are:
▁கருதப்பட்ட -0.0
▁க

In [101]:
for _ in range(10):
  random_word_similarity()
  print()

Words most similar to: ▁இப் are:
▁இப் -0.0
▁இத் -7.939371872047522
▁இந்தப் -8.831170980768933
▁இக் -8.879538941055483
▁இந்தக் -9.172147763400185
▁இம் -9.267071564659924
▁இந் -9.39694580089282
▁இச் -9.459149058084623
▁இந்தத் -9.689517942806392
▁இவ் -9.724935199228561
▁இவ்வ -9.736857521091688
▁இதற்கு -9.81011843309302
▁இ -9.853839151583491
▁இவ்வி -10.026034936552515
▁இப்பகுதி -10.09364578133977

Words most similar to: 34 are:
34 -0.0
56 -8.643708780186127
53 -8.73851306732013
33 -8.7464346480265
44 -8.840609026174928
45 -9.043567534909178
37 -9.2644904842707
29 -9.272244976412193
49 -9.30807528745771
32 -9.381125035189395
25 -9.425149704573172
39 -9.459492338807943
42 -9.494180305268264
48 -9.498234810150045
27 -9.5798010086725

Words most similar to: ▁பயிற்சி are:
▁பயிற்சி -0.0
பயிற்சி -8.95039181964132
▁திறமை -9.04297586656306
▁பயிற்ற -9.292198763393287
▁ஆலோசனை -9.360545629188243
="270 -9.541765364751969
▁ஆராய்ச்சி -9.78282284002629
▁உடற்பயிற்சி -9.818646717793152
=124 -9.8201708783123